In [1]:
import hydraseq as hds
from hydraseq import run_convolutions

In [2]:
from collections import namedtuple, defaultdict
Convo = namedtuple('Convo', ['start', 'end', 'pattern', 'lasts', 'nexts'])
endcap = Convo(-1,-1,['end'], [],[])
def to_convo_node(lst_stuff):
    return Convo(lst_stuff[0], lst_stuff[1], lst_stuff[2], [], [])

def link(conv1, conv2):
    conv1.nexts.append(conv2)
    conv2.lasts.append(conv1)

def stackem(lst_convos):
    frame = defaultdict(list)
    ends = []
    for convo in lst_convos:
        if frame[convo[0]]:
            for cnode in frame[convo[0]]:
                convo_node = to_convo_node(convo)
                link(cnode, convo_node)
                ends.append(convo_node)
                if cnode in ends: ends.remove(cnode)
                frame[convo_node.end].append(convo_node)
        else:
            convo_node = to_convo_node(convo)
            ends.append(convo_node)
            frame[convo_node.end].append(convo_node)
    return ends

def recon(end_nodes):
    stack = []
    for node in end_nodes:
        sentence = []
        sentence.append([node.start, node.end, node.pattern])
        while node.lasts:
            node = node.lasts[0]
            sentence.append([node.start, node.end, node.pattern])
        sentence.reverse()
        stack.append(sentence)
    return stack

def pats_only(sentence):
    return [sent[2] for sent in sentence]

def get_init_sentence_from_hydra(hd0):
    sentence = []
    node = hd0.n_init.nexts[0]
    idx = 0
    while node.nexts:
        sentence.append([idx, idx+1, [node.key]])
        node = node.nexts[0]
        idx+=1
    return [sentence]


def run_them_all(sentences, hd1):
    next_sentences = []
    for sent in sentences:
        conv = run_convolutions(pats_only(sent), hd1)
        for item in recon(stackem(conv)):
            next_sentences.append(item)
    return next_sentences

def think(lst_hydras):
    active_layers = []
    for idx, hydra in enumerate(lst_hydras):
        sentences = run_them_all(sentences, hydra) if idx != 0 else get_init_sentence_from_hydra(hydra)
        active_layers.append(sentences)
    return active_layers


In [3]:
hd0 = hds.Hydraseq("0_")
hd0.insert("spring leaves spring END")

uuid: 0_
n_init: <node: (*),(*)>:
active values: ['END']
next values: []

In [4]:
hd1 = hds.Hydraseq("1_")
encodes = [
    "spring 1_Adj",
    "spring 1_Verb",
    "spring 1_Noun",
    "leaves 1_Noun",
    "leaves 1_Verb"
]
for encode in encodes:
    hd1.insert(encode)

In [5]:
hd2 = hds.Hydraseq('_')
for pattern in [
    "1_Noun _NP_",
    "1_Adj 1_Noun _NP_",
    "1_Verb _VP_",
    "1_Adj 1_Noun _NP_",
    "1_Adj 1_Adj 1_Adj 1_Noun _NP_",

]:
    hd2.insert(pattern)

In [6]:
sentences = get_init_sentence_from_hydra(hd0)
sentences

[[[0, 1, ['spring']], [1, 2, ['leaves']], [2, 3, ['spring']]]]

In [7]:
hd1 = hds.Hydraseq('1_')
for pattern in [
    "o 1_eye",
    "L 1_nose",
    "m 1_mouth",
    "sdfg 1_keys",
]:
    hd1.insert(pattern)

hd2 = hds.Hydraseq('2_')
for pattern in [
    "1_eye 1_eye 2_eyes",
    "1_nose 2_nose",
    "1_mouth 2_mouth",
    "1_keys 1_keys 1_keys 2_row"
]:
    hd2.insert(pattern)

hd3 = hds.Hydraseq('3_')    
for pattern in [
    "2_eyes 2_nose 2_mouth 3_face",
    "2_row 3_homerow"
]:
    hd3.insert(pattern)

In [8]:
hd0 = hds.Hydraseq("_")
hd0.insert("x x o o L m end sdfg sdfg sdfg sdfg _period")
think([hd0, hd1, hd2, hd3])



[[[[0, 1, ['x']],
   [1, 2, ['x']],
   [2, 3, ['o']],
   [3, 4, ['o']],
   [4, 5, ['L']],
   [5, 6, ['m']],
   [6, 7, ['end']],
   [7, 8, ['sdfg']],
   [8, 9, ['sdfg']],
   [9, 10, ['sdfg']],
   [10, 11, ['sdfg']]]],
 [[[2, 3, ['1_eye']],
   [3, 4, ['1_eye']],
   [4, 5, ['1_nose']],
   [5, 6, ['1_mouth']]],
  [[7, 8, ['1_keys']],
   [8, 9, ['1_keys']],
   [9, 10, ['1_keys']],
   [10, 11, ['1_keys']]]],
 [[[0, 2, ['2_eyes']], [2, 3, ['2_nose']], [3, 4, ['2_mouth']]],
  [[0, 3, ['2_row']]],
  [[1, 4, ['2_row']]]],
 [[[0, 3, ['3_face']]], [[0, 1, ['3_homerow']]], [[0, 1, ['3_homerow']]]]]